# CSCI 3202, Spring 2025
# Final Project
# 100 Points
# Due: March 21 at 11:59 pm

<br> 

### Your name:Tanmay Meti and Justin Goh

<br> 

---

## Mancala rules for this homework assignment
**(there are many different rules sets for Mancala.  Please read this before writing the code)**

* Players sit on opposite sides of the long edge of the board
* There are 6 small pits in the middle of the board and 2 large ones at each end.  The small ones in the middle and the large pit on your right are yours.  The small ones on the other side and the large pit to your opponent's right are theirs
* The large pits at the end of the board are called Mancalas
* Set up the board with 4 stones per small pit (none in the mancalas)
* On every turn, select a pit on your side of the board that contains one or more stones,  then distribute its stones, one stone per pit, in an counter-clockwise direction until you have no stones remaining
* If you encounter your opponent's mandala, skip it
* If you encounter your mancala, drop a stone into it
* If the last stone lands in an empty pit on your side of the board, capture this stone and any stones in your opponent's pit on the other side of the board, collect all of these stones, including the one that just landed, and place them into your mancala.
* If either player's pits are entirely empty, the game concludes. 
* The player who still has stones on his side of the board when the game concludes places all of these pieces into their mancala.
The player with the most stones in their mancala is declared the winner. If both players have an equal number of stones in their mancala, the game results in a tie.


In [ ]:
import games4e
import random
import copy
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        pit -= 1
        if self.current_player == 1:
            if 0 <= pit <= self.p1_pits_index[1] and self.board[pit] > 0:
                return True
        elif self.current_player == 2:
            real_index = pit + self.p1_mancala_index + 1
            if self.p2_pits_index[0] <= real_index <= self.p2_pits_index[1] and self.board[real_index] > 0:
                return True
        return False
        pass
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        valid_pits = []
        if self.current_player == 1:
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                if self.board[i] > 0:
                    valid_pits.append(i + 1)
        else:
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                if self.board[i] > 0:
                    valid_pits.append(i - self.p1_mancala_index)
        return random.choice(valid_pits) if valid_pits else None
        pass
    
    def play(self, pit):
        """Execute a move and switch turns unconditionally (no extra turns)"""
        if self.winning_eval():
            print("GAME OVER")
            return self.board
        
        # Validate move
        pit -= 1  # Convert to 0-based index
        if self.current_player == 1:
            if not (0 <= pit <= self.p1_pits_index[1] and self.board[pit] > 0):
                print("INVALID MOVE")
                return self.board
            mancala = self.p1_mancala_index
            opponent_mancala = self.p2_mancala_index
        else:
            real_index = pit + self.p1_mancala_index + 1
            if not (self.p2_pits_index[0] <= real_index <= self.p2_pits_index[1] 
                    and self.board[real_index] > 0):
                print("INVALID MOVE")
                return self.board
            pit = real_index
            mancala = self.p2_mancala_index
            opponent_mancala = self.p1_mancala_index

        # Distribute stones
        stones = self.board[pit]
        self.board[pit] = 0
        current_index = pit
        
        while stones > 0:
            current_index = (current_index + 1) % len(self.board)
            if current_index == opponent_mancala:  # Skip opponent's mancala
                continue
            self.board[current_index] += 1
            stones -= 1

        # Capture rule
        if (self.current_player == 1 and 
            0 <= current_index <= self.p1_pits_index[1] and 
            self.board[current_index] == 1):
            
            opposite_index = self.p2_pits_index[1] - (current_index - self.p1_pits_index[0])
            captured = self.board[opposite_index]
            if captured > 0:
                self.board[self.p1_mancala_index] += captured + 1
                self.board[opposite_index] = 0
                self.board[current_index] = 0

        elif (self.current_player == 2 and 
            self.p2_pits_index[0] <= current_index <= self.p2_pits_index[1] and 
            self.board[current_index] == 1):
            
            opposite_index = self.p1_pits_index[1] - (current_index - self.p2_pits_index[0])
            captured = self.board[opposite_index]
            if captured > 0:
                self.board[self.p2_mancala_index] += captured + 1
                self.board[opposite_index] = 0
                self.board[current_index] = 0

        # Always switch players (modified rule: no extra turns)
        self.current_player = 2 if self.current_player == 1 else 1
        self.moves.append((self.current_player, pit + 1))
        
        return self.board
    def winning_eval(self, print_result=False):
        """
        Function to verify if the game board has reached the winning state.
        print_result: boolean to control whether to print the winner
        """
        p1_empty = all(self.board[i] == 0 for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
        p2_empty = all(self.board[i] == 0 for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))
        
        if p1_empty or p2_empty:
            self.board[self.p1_mancala_index] += sum(self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1])
            self.board[self.p2_mancala_index] += sum(self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1])
            
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                self.board[i] = 0
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                self.board[i] = 0
                
            if print_result:  
                if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                    print("Player 1 wins!")
                elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                    print("Player 2 wins!")
                else:
                    print("It's a tie!")
            return True
        return False
    def evaluate_v2(self):
        p1_store = self.board[self.p1_mancala_index]
        p2_store = self.board[self.p2_mancala_index]
        p1_side = sum(self.board[i] for i in range(self.p1_pits_index[0], self.p1_pits_index[1]+1))
        p2_side = sum(self.board[i] for i in range(self.p2_pits_index[0], self.p2_pits_index[1]+1))
        base = (p1_store - p2_store) + 0.5*(p1_side - p2_side)
        bonus = 0
        if self.moves:
            last_player, last_pit = self.moves[-1]
        if last_player == 1 and self.current_player == 1 and last_pit == self.p1_mancala_index:
            bonus = 2
        elif last_player == 2 and self.current_player == 2 and last_pit == self.p2_mancala_index:
            bonus = -2
        return base + bonus


In [ ]:
def simple_simulation_100_games():
    random.seed(35) 

    results = []  

    for i in range(100):
        game = Mancala()
        while not game.winning_eval():
            if game.current_player == 1:
                valid_pits = [i + 1 for i in range(game.pits_per_player) if game.board[i] > 0]
            else:
                valid_pits = [i + 1 for i in range(game.pits_per_player) 
                              if game.board[i + game.pits_per_player + 1] > 0]
            
            if not valid_pits:
                break  
            
            chosen_pit = random.choice(valid_pits)
            game.play(chosen_pit)
        
        if game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]:
            results.append(1)  
        elif game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]:
            results.append(2) 
        else:
            results.append(0)  

    p1_wins = results.count(1)
    p2_wins = results.count(2)
    ties = results.count(0)
    
    p1_win_pct = (p1_wins / 100) * 100
    p2_win_pct = (p2_wins / 100) * 100
    tie_pct = (ties / 100) * 100

    print(f"Player 1 wins: {p1_wins} ({p1_win_pct:.1f}%)")
    print(f"Player 2 wins: {p2_wins} ({p2_win_pct:.1f}%)")
    print(f"Ties: {ties} ({tie_pct:.1f}%)")
simple_simulation_100_games()




Player 1 wins: 48 (48.0%)
Player 2 wins: 45 (45.0%)
Ties: 7 (7.0%)


In [3]:
def play_human_game():
    game = Mancala()
    print("=== Mancala — Human vs Human ===")
    while not game.winning_eval():
        game.display_board()
        try:
            pit = int(input(f"Player {game.current_player} — choose a pit (1‑{game.pits_per_player}): "))
        except ValueError:
            print("Please enter a number.")
            continue
        if game.valid_move(pit):
            game.play(pit)
        else:
            print("Illegal pit: choose a non‑empty pit on your side.")
    game.display_board()
    game.winning_eval(print_result=True)

In [ ]:
play_human_game()

In [5]:
def random_vs_random(num_games=100, seed=1):
    random.seed(seed)
    p1_wins = p2_wins = ties = total_moves = 0
    for i in range(num_games):
        g = Mancala()
        moves = 0
        while not g.winning_eval():
            if g.current_player == 1:
                g.play(g.random_move_generator())
            else:
                g.play(g.random_move_generator())
            moves += 1
        total_moves += moves
        if g.board[g.p1_mancala_index] > g.board[g.p2_mancala_index]:
            p1_wins += 1
        elif g.board[g.p2_mancala_index] > g.board[g.p1_mancala_index]:
            p2_wins += 1
        else:
            ties += 1
    print(f"P1 wins: {p1_wins}  ({p1_wins/num_games*100:.1f}%)")
    print(f"P2 wins: {p2_wins}  ({p2_wins/num_games*100:.1f}%)")
    print(f"Ties   : {ties}  ({ties/num_games*100:.1f}%)")
    print(f"Avg moves/game: {total_moves/num_games:.1f}")

In [6]:
random_vs_random()

P1 wins: 52  (52.0%)
P2 wins: 42  (42.0%)
Ties   : 6  (6.0%)
Avg moves/game: 43.0


In [7]:
import numpy as np
import copy
def alpha_beta_search_mancala(state, d=5):
    """Alpha-beta search for Mancala, working directly with the Mancala class"""
    player = state.current_player
    
    def cutoff_test(state, depth):
        return depth > d or state.winning_eval()
    
    def get_utility(state):
        """Calculate utility based on stones in Mancalas"""
        if player == 1:
            return state.board[state.p1_mancala_index] - state.board[state.p2_mancala_index]
        else:
            return state.board[state.p2_mancala_index] - state.board[state.p1_mancala_index]

    def get_valid_moves(state):
        """Get valid moves for current player"""
        valid_moves = []
        if state.current_player == 1:
            for i in range(state.p1_pits_index[0], state.p1_pits_index[1] + 1):
                if state.board[i] > 0:
                    valid_moves.append(i + 1)
        else:
            for i in range(state.p2_pits_index[0], state.p2_pits_index[1] + 1):
                if state.board[i] > 0:
                    valid_moves.append(i - state.p1_mancala_index)
        return valid_moves

    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return get_utility(state)
        v = -np.inf
        for a in get_valid_moves(state):
            next_state = copy.deepcopy(state)
            next_state.play(a)
            v = max(v, min_value(next_state, alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return get_utility(state)
        v = np.inf
        for a in get_valid_moves(state):
            next_state = copy.deepcopy(state)
            next_state.play(a)
            v = min(v, max_value(next_state, alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Find best move
    best_score = -np.inf
    beta = np.inf
    best_action = None
    
    for a in get_valid_moves(state):
        next_state = copy.deepcopy(state)
        next_state.play(a)
        v = min_value(next_state, best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
            
    return best_action

In [8]:
class MancalaAI:
    def __init__(self, player_number, depth=5):
        self.player_number = player_number
        self.depth = depth
    
    def get_move(self, state):
        """Returns the best move using the improved alpha-beta search."""
        return alpha_beta_search_mancala(state, self.depth)

In [9]:
import random
import time
import numpy as np
def simulate_alpha_beta_vs_random(num_games=100, depth=5, seed=42):
    random.seed(seed)
    ai_wins = rand_wins = ties = 0
    total_moves = 0
    total_time  = 0.0
    for i in range(num_games):
        g   = Mancala()
        ai  = MancalaAI(player_number=1, depth=depth)
        moves_this_game = 0
        start = time.perf_counter()
        while not g.winning_eval():
            if g.current_player == 1:
                g.play(ai.get_move(g))
            else:
                legal = [i - g.p1_mancala_index
                         for i in range(g.p2_pits_index[0], g.p2_pits_index[1]+1)
                         if g.board[i] > 0]
                if not legal:                
                    break
                g.play(random.choice(legal))
            moves_this_game += 1
        total_time  += time.perf_counter() - start
        total_moves += moves_this_game
        p1_score = g.board[g.p1_mancala_index]
        p2_score = g.board[g.p2_mancala_index]
        if p1_score > p2_score:
            ai_wins += 1
        elif p2_score > p1_score:
            rand_wins += 1
        else:
            ties += 1
    print(f"Alpha‑Beta wins:  {ai_wins}/{num_games}  ({ai_wins/num_games*100:.1f} %)")
    print(f"Random wins:     {rand_wins}/{num_games}  ({rand_wins/num_games*100:.1f} %)")
    print(f"Ties:            {ties}/{num_games}      ({ties/num_games*100:.1f} %)")
    print(f"Avg moves/game:  {total_moves/num_games:.1f}")
    print(f"Avg seconds/game:{total_time/num_games:.4f}")
#simulate_alpha_beta_vs_random()

In [10]:
import copy
def minimax_search_mancala(state, depth_limit=5):
    player = state.current_player
    def cutoff_test(node, depth):
        return depth >= depth_limit or node.winning_eval()
    def utility(node):
        if player == 1:
            return node.board[node.p1_mancala_index] - node.board[node.p2_mancala_index]
        else:
            return node.board[node.p2_mancala_index] - node.board[node.p1_mancala_index]
    def valid_moves(node):
        moves = []
        if node.current_player == 1:
            pits = range(node.p1_pits_index[0], node.p1_pits_index[1] + 1)
            moves = [i + 1 for i in pits if node.board[i] > 0]
        else:
            pits = range(node.p2_pits_index[0], node.p2_pits_index[1] + 1)
            moves = [i - node.p1_mancala_index for i in pits if node.board[i] > 0]
        return moves
    def max_value(node, depth):
        if cutoff_test(node, depth):
            return utility(node)
        v = -np.inf
        for move in valid_moves(node):
            child = copy.deepcopy(node)
            child.play(move)
            v = max(v, min_value(child, depth + 1))
        return v
    def min_value(node, depth):
        if cutoff_test(node, depth):
            return utility(node)
        v = np.inf
        for move in valid_moves(node):
            child = copy.deepcopy(node)
            child.play(move)
            v = min(v, max_value(child, depth + 1))
        return v
    best_score = -np.inf
    best_move  = None
    for move in valid_moves(state):
        child = copy.deepcopy(state)
        child.play(move)
        score = min_value(child, 1)
        if score > best_score:
            best_score, best_move = score, move
    return best_move

In [11]:
class MinimaxAI:
    def __init__(self, player_number, depth=5):
        self.player_number = player_number
        self.depth = depth
    def get_move(self, state):
        return minimax_search_mancala(state, self.depth)

In [12]:
test_game = Mancala()
mm_ai = MinimaxAI(player_number=1, depth=3)
move = mm_ai.get_move(test_game)
print("Minimax recommends pit:", move)

Minimax recommends pit: 6


In [13]:
def simulate_minimax_vs_random(num_games=100, depth=5):
    mm_wins = rand_wins = ties = total_moves = 0
    for i in range(num_games):
        g = Mancala()
        mm = MinimaxAI(player_number=1, depth=depth)
        moves_this_game = 0 
        while not g.winning_eval():
            if g.current_player == 1:
                g.play(mm.get_move(g))
            else:
                legal = [i - g.p1_mancala_index for i in range(g.p2_pits_index[0],g.p2_pits_index[1] + 1)
                         if g.board[i] > 0]
                g.play(random.choice(legal))
            moves_this_game += 1
        total_moves += moves_this_game
        if g.board[g.p1_mancala_index] > g.board[g.p2_mancala_index]:
            mm_wins += 1
        elif g.board[g.p1_mancala_index] < g.board[g.p2_mancala_index]:
            rand_wins += 1
        else:
            ties += 1
    print(f"Minimax wins:  {mm_wins} ({mm_wins/num_games*100:.1f}%)")
    print(f"Random wins:   {rand_wins} ({rand_wins/num_games*100:.1f}%)")
    print(f"Ties:          {ties} ({ties/num_games*100:.1f}%)")
    print(f"Avg moves/game: {total_moves/num_games:.1f}")
#simulate_minimax_vs_random()

In [ ]:
Mancala.evaluate = Mancala.evaluate_v2     
simulate_alpha_beta_vs_random(num_games=100, depth=10, seed=11)